In [10]:
import math

from IPython.display import display, Math, Latex

# Libraries you might need
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import random
import numpy as np
import sklearn.metrics

# Homework 6

## Exercise 1

Let random variable $X$ be a discrete random variable with values
{−2, −1, 0, 1, 2}, obtained with the same probability $p = \tfrac15$. Let
the second random variable $Y$ be $Y = X^2$.

(a) Compute analytically $cov(X, Y)$ (2 points)

(b) Compute the mutual information between X and Y in bits (2 points).

(c) Simulate 100, 1000, 10000 data realization of these processes and com-
pute the covariances and mutual information based on the data. (2 points)

*+1 bonus point if you compute the mutual information both with a library and yourself with using the formula.*

**Answer (a)**

$$
E[X] = \sum_i x_i \cdot P(X = x_i) = \frac{1}{5} \cdot (−2 − 1 + 0 + 1 + 2) = 0
$$

$$
E[Y] = E[X^2] = \sum_i x_i^2 \cdot P(X = x_i) = \frac{1}{5} \cdot ((−2)^2 + (−1)^2 + 0^2 + 1^2 + 2^2) = \frac{1}{5} \cdot 10 = 2
$$

$$
E[XY] = E[X \cdot X^2] = E[X^3] = \sum_i x_i^3 \cdot P(X = x_i) = \frac{1}{5} \cdot ((−2)^3 + (−1)^3 + 0^3 + 1^3 + 2^3) = 0
$$

$$
cov(X, Y) = E[XY] - E[X]E[Y] = 0 − (0 \cdot 2) = 0
$$


**Answer (b)**

$$
I(X, Y) = \sum_{x \in \mathcal{A}_X} \sum_{y \in \mathcal{A}_Y} p_{(X, Y)}(x, y) \cdot log_2 \frac{p_{(X, Y)}(x, y)}{p_X(x) \cdot p_Y(y)}
$$

For each pair $(x, y)$: $p_{(X, Y)}(x, y) = \frac{1}{5}$

Table. Probabilities of $Y$.

| $y$ | $P(Y = y)$    |
|----|---------------|
| 4  | $\frac{2}{5}$ |
| 1  | $\frac{2}{5}$ |
| 0  | $\frac{1}{5}$ |

\begin{equation*}
\begin{split}
I(X, Y) &= \sum_{x \in \mathcal{A}_X} \sum_{y \in \mathcal{A}_Y} p_{(X, Y)}(x, y) \cdot log_2 \frac{p_{(X, Y)}(x, y)}{p_X(x) \cdot p_Y(y)} \\
&= \frac{1}{5} \cdot \big{(} 4 \cdot log_2 \frac{ \frac{1}{5} }{\frac{2}{5} \cdot \frac{1}{5}} + log_2 \frac{ \frac{1}{5} }{\frac{1}{5} \cdot \frac{1}{5}} \big{)} \\
&= \frac{1}{5} \cdot \big{(} 4 \cdot log_2 \frac{5}{2} + log_2\ 5\big{)} \\
&= \frac{1}{5} \cdot \big{(} 4 \cdot (log_2\ 5 - log_2\ 2) + log_2\ 5\big{)} \\
&= \frac{1}{5} \cdot \big{(} 4 \cdot log_2\ 5 - 4 + log_2\ 5\big{)} \\
&= \frac{1}{5} \cdot \big{(} 5 \cdot log_2\ 5 - 4 \big{)} \\
&= log_2\ 5 - \frac{4}{5} \text{[bits]}
\end{split}
\end{equation*}

In [49]:
def probability(x, bins):
    return np.histogram(x, bins = bins)[0] / len(x)

In [50]:
def joint_probability(x, y, binsx, binsy):
    counts = np.histogram2d(x, y, bins = [binsx, binsy])[0].flatten()
    return counts[ counts != 0 ] / len(x)

In [51]:
def entropy(probs):
    return -np.sum([p * np.log2(p) if p > 0 else 0 for p in probs])

In [53]:
def mutual_information(x, y):
    binsx = [-2, -1, 0, 1, 2, 3]
    binsy = [0, 1, 4, 5]
    
    px = probability(x, bins = binsx)
    entropy_x = entropy(px)

    py = probability(y, bins = binsy)
    entropy_y = entropy(py)

    pxy = joint_probability(x, y, binsx, binsy)
    entropy_xy = entropy(pxy)

    return entropy_x + entropy_y - entropy_xy

In [72]:
def random_x(n = 1):
    return [ random.choice([ -2, -1, 0, 1, 2 ]) for _ in range(n) ]

In [73]:
random.seed(384593)

for n_trials in [100, 1000, 10_000]:
    x = random_x(n = n_trials)
    y = np.power(x, 2)
    
    print(f"N         = {n_trials}")
    print(f"Cov       = {np.cov(x, y)[0][1]}")
    print(f"I_formula = {mutual_information(x, y)}")
    print(f"I_sklearn = {sklearn.metrics.mutual_info_score(x, y) / np.log(2)}")
    print("")

N         = 100
Cov       = 0.18181818181818182
I_formula = 1.4869224242853196
I_sklearn = 1.4869224242853205

N         = 1000
Cov       = -0.03702502502502533
I_formula = 1.5173387324484375
I_sklearn = 1.517338732448437

N         = 10000
Cov       = -0.021542484248424816
I_formula = 1.5231969968482622
I_sklearn = 1.523196996848264



---
## Exercise 2
Suppose that we have a neuron which, in a given time period, will fire
with probability 0.1, yielding a Bernoulli distribution for the neuron’s firing (denoted by the random variable R = 0 or 1) with p(R = 1) = 0.2.

(a) Compute the entropy H(R) of this distribution (calculated in bits, i.e., using the base 2 logarithm)? (1 point)

(b) Now lets add a stimulus to the picture. Suppose that we think this
neuron's activity is related to a light flashing in the eye. Let us say that
the light is flashing in a given time period with probability 0.2. Call this
stimulus random variable S. If there is a flash, the neuron will fire with
probability 1/2. If there is no flash, the neuron will fire with probability
1/8. Call the random variable describing whether the neuron fires or not
R. Compute the mutual information I(S : R)? (2 points)\
*Hint: First, confirm that H(R) is the same as above by computing p(R).*

**Answer (a)**


**Answer (b)**
